### Описание данных в файле transactions.csv

* customer_id - идентификатор клиента
* tr_datetime - день и время совершения транзакции (дни нумеруются с начала данных)
* mcc_code - mcc-код транзакции
* tr_type - тип транзакции
* amount - сумма транзакции в условных единицах; со знаком "+" — начисление средств клиенту (приходная транзакция), "-" — списание средств (расходная транзакция)
* term_id - идентификатор терминала

### Описание задания

Цель задания выполнить последовательно все упражнения. Будет оцениваться правильность кода, и конечный результат, т.е. после прогона всех ячеек должен получится преобразованный датасет в файле features.csv.

Обратите внимание, что задания можно выполнить разными способами, конретное решение не навязывается, однако код должен быть по возможности хорошо читаемым и лаконичным.

**Хочу отметить**: я постараюсь сделать в двух вариантах в pandas и в spark.

**13.04** pandas done.

**13.04** spark wait.

### 1. Создать sql context

https://spark.apache.org/docs/1.6.1/sql-programming-guide.html#starting-point-sqlcontext

In [1]:
#Initializing PySpark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

In [ ]:
#Spark Config
conf = SparkConf().setAppName("sample_app")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/15 12:25:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
sqlContext = SQLContext(sc)

df = sqlContext.read.csv('transactions.csv',
                         # header=True)

# Displays the content of the DataFrame to stdout
df.show()

### 2. Создать DataFrame из файла transactions.csv

Хотя выше мы уже создали DF создадим его еще двумя способами: 
* через spark
* через pandas

In [ ]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import SparkSession

у spark решил сделать еще одни способом 

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Создадим DF:
example = spark.read.csv('transactions.csv', header=True)
example.show()

pandas ниже

In [ ]:
data = pd.read_csv('transactions.csv')

In [ ]:
data.head()

In [ ]:
data.info()

### 3. Напечатать схему

In [ ]:
# df.printSchema()

In [ ]:
data.info()

у spark изначально не подгружаются типы, везде string. У pandas типы данных указаны, стоит изменить :
* tr_datetime to datetime
* customer_id - это не совсем числовое значение , это категориальное , так как мы не можем делать операции мат. с ним.
* mcc_code, tr_type, term_id - тоже object.
* c amount нормально

для изменения в spark типов данных https://www.geeksforgeeks.org/update-pyspark-dataframe-metadata/

### 4. Отобразить первые 20 строк DataFrame-а

In [ ]:
data.head(20)

In [ ]:
df.show(20)

### 5. Посчитать количество уникальных customer_id

In [ ]:
data['customer_id'].nunique()

In [ ]:
df.select('customer_id').distinct().count()

### 6. Посчитать количество уникальных term_id

In [ ]:
data['term_id'].nunique()

### 7. Посчитать среднее количество транзакций на одного customer_id

так как кол-во трнзакций целочисленное число, то нужно округлить

In [ ]:
round(data.groupby(['customer_id']).agg({'tr_datetime':'count'}).mean().iloc[0])

In [ ]:
total_count = int(len(data)/data['customer_id'].nunique())

In [ ]:
print(f'Средняя кол-во транзакций на 1 пользователя: {total_count}')

### 8. Посчитать среднюю сумму транзакций на одного customer_id

* исходя из условия, что '-' это списание средств, а '+' это начиление клиенту.

Если имелось ввиду всех транзакций (суммы их), то нам нужно взять модуль от суммы прихода и списания (сложить их) и поделить на кол-во id.

In [ ]:
mean_transaction = abs(data['amount'].sum())/data['customer_id'].nunique()
print(f'Средняя сумма в уе на 1 пользователя: {mean_transaction}')

### 9. Удалить столбец term_id

In [ ]:
data = data.drop('term_id', axis=1)

In [ ]:
data

### 10. Добавить столбец direction, который указывает "направление" транзакции, если в поле amount отрицательное значение то туда записать D, если положительное - C

In [ ]:
data['direction'] = data['amount'].apply(lambda x: 'C' if x>0 else 'D') 

### 11. Столбец amount преобразовать в абсолютное значение

In [ ]:
data['amount'] = abs(data['amount'])

### 12. Посчитать среднюю сумму транзакций на одного customer_id отдельно по каждому направлению

In [ ]:
data_direction = data.groupby(['direction'])\
.agg({'amount':'sum', 
      'customer_id':'nunique'})

data_direction['mean'] = data_direction['amount']/data_direction['customer_id']

In [ ]:
data_direction

In [ ]:
data['customer_id'].nunique()

In [ ]:
data_direction2 =pd.pivot_table(data, 
               index= 'direction', 
               values =['amount','customer_id'], 
               aggfunc={'amount':'sum', 
                        'customer_id':'nunique'})


data_direction2['mean'] = data_direction2['amount']/data_direction2['customer_id']

In [ ]:
c_dir = data_direction['mean'].loc['C']
d_dir = data_direction['mean'].loc['D']

In [ ]:
print(f"""Средняя сумма в уе на 1 пользователя в {data_direction.index[0]}: {c_dir}
Средняя сумма в уе на 1 пользователя в {data_direction.index[1]}: {d_dir}
"""
)

### 13, 14 задания удалил, были криво сформулированы

ok.

### 15. Сделать pivot, в котором строки это customer_id, столбцы mcc-коды, в ячейках суммы по amount

In [ ]:
data_mcc = data.pivot_table(index='customer_id', 
                 columns='mcc_code',
                 values='amount', 
                 aggfunc='sum')\
.fillna(0)

data_mcc

In [ ]:
data_mcc.reset_index(inplace=True)

In [ ]:
data_mcc

### 16. Сделать pivot, в котором строки это customer_id, столбцы mcc-коды, в ячейках средние и стандартные отклонения по amount
т.е. на каждый mcc_code должно быть до 2-х столбцов со средним и стандартным отклонением

In [ ]:
data_mcc_2 = data.pivot_table(index='customer_id', 
                 columns='mcc_code',
                 values=['amount'], 
                 aggfunc=['mean','std'])\
.fillna(0)

data_mcc_2

In [ ]:
# Найдем значения, которые не были в стандартном отклонении
set(data_mcc_2['mean'].columns) - set(data_mcc_2['std'].columns)

In [ ]:
# '742_mcc_avg'
data_mcc_2.columns = [f"{i[2]}_mcc_{i[0]}" for i in data_mcc_2.columns]
data_mcc_2.reset_index(inplace=True)

In [ ]:
data_mcc_2

Мне захотелось осортировать столбцы, но в то же время важно было оставить 1 столбцом customer_id, поэтому:

In [ ]:
mcc = pd.concat([data_mcc_2['customer_id'], 
           data_mcc_2.iloc[:,1:].reindex(sorted(data_mcc_2.iloc[:,1:].columns), axis=1)], 
          axis=1)
mcc

оказаось 361 столбцец, хотя ожидалосб 367. Посмотрим почему.

In [ ]:
data_mcc_2

In [ ]:
# в 6 значениях mcc_code стандартное отлконение не выдает, 
# это по ходу из-за особенностей высчиления через pivot_table в pandas,а также, что у каждого из значений по 1 объекту
# по сути должен быть 0, более вероятноиз-за формулы. в знаменателе n-1 чаще всего, хотя здеcь считается нормально.
data[data['mcc_code'] == 6513].iloc[0]['amount'].std()

In [ ]:
data[data['mcc_code'] == 6513]

In [ ]:
import numpy as np

In [ ]:
data[data['mcc_code'] == 6513].pivot_table(index='customer_id', 
                 columns='mcc_code',
                 values=['amount'], 
                 aggfunc=['std'])\
.fillna(0)

### 17. Сделать pivot, в котором строки это customer_id, столбцы типы транзакций, в ячейках средние и стандартные отклонения по amount, значения должны быть разделены по направлениям
т.е. на каждый tr_type должно быть до 4-х столбцов со средним и стандартным отклонением по каждому направлению

**Подсказка:** Можно сделать расчеты отдельно для каждого направления платежей, потом присоединить к заранее подготовленному списку уникальных customer_id. Так будет проще, наглядней и меньше вероятность сделать ошибку.

In [ ]:
data_type = data.pivot_table(index='customer_id', 
                 columns=['tr_type','direction'],
                 values=['amount'], 
                 aggfunc=['mean','std'])\
.fillna(0)

data_type

In [ ]:
# '1000_d_type_avg'
data_type.columns = [f"{i[2]}_{i[-1]}_{'type'}_{i[0]}" for i in data_type.columns]
data_type.reset_index(inplace=True)
data_type

In [ ]:
type = pd.concat([data_type['customer_id'], 
           data_type.iloc[:,1:].reindex(sorted(data_type.iloc[:,1:].columns), axis=1)], 
          axis=1)
type

### 18. Извлечь часы из столбца tr_datetime и удалить столбец tr_datetime

Есть несколько решений данной задачи:
1) перевод в datetime и взять оттуда часы. Но выдает ошибку при переводе в pd.to_datetime 'DateParseError: second must be in 0..59: 59 12:29:60, at position 7' Это нужно доп. функцию писать, чтобы от 60 секунд избавиться.
2) взять по индексу эл-ов через функцию :)

In [ ]:
data['hours'] = data['tr_datetime'].apply(lambda x: x[-8:-6])

In [ ]:
data['hours'] = data['hours'].astype('int')

In [ ]:
# проверим нет ли выбросов, как с секундами
data['hours'].value_counts().sort_index()

In [ ]:
data = data.drop('tr_datetime', axis=1)

In [ ]:
data

### 19. Сделать pivot, в котором строки это customer_id, столбцы часы, полученные на предыдущем этапе, в ячейках средние и стандартные отклонения по amount, значения должны быть разделены по направлениям

**Подсказка:** Можно сделать расчеты отдельно для каждого направления платежей, потом присоединить к заранее подготовленному списку уникальных customer_id. Так будет проще, наглядней и меньше вероятность сделать ошибку.

In [ ]:
data

In [ ]:
data_hours = data.pivot_table(index='customer_id', 
                 columns=['hours','direction'],
                 values=['amount'], 
                 aggfunc=['mean','std'])\
.fillna(0)

data_hours

In [ ]:
# '0_hour_c_avg'
data_hours.columns = [f"{i[2]}_hour_{i[-1]}_{i[0]}" for i in data_hours.columns]
data_hours.reset_index(inplace=True)
data_hours

In [ ]:
hours = pd.concat([data_hours['customer_id'], 
           data_hours.iloc[:,1:].reindex(sorted(data_hours.iloc[:,1:].columns), axis=1)], 
          axis=1)
hours

### 20. Соединить полученный DataFrame с pivot-ом по mcc кодам и по часам

**Примечание:** Суть тут в том, что мы по формируем набор данных, где для каждого customer_id мы имеем рассчитанные на основе транзакций признаки, такие как среднее арифметическое и стандартное отклонение сумм транзакций для каждого mcc кода, для каждого mcc с учетом направления транзакции, и для каждого часа в сутках без mcc кодов, но с учетом направления транзакции.

**Подсказка:** Список полей результирующего набора данных(… - другие аналогичные поля):

        ['customer_id',
         '742_mcc_avg',
         '742_mcc_std',
         '1711_mcc_avg',
         '1711_mcc_std',
         '1731_mcc_avg',
         '1731_mcc_std',
         ...
         ...
         ...
         '1010_c_type_avg',
         '1010_c_type_std',
         '1030_c_type_avg',
         '1030_c_type_std',
         '1100_c_type_avg',
         ...
         ...
         '1000_d_type_avg',
         '1000_d_type_std',
         '1010_d_type_avg',
         '1010_d_type_std',
         '1030_d_type_avg',
         ...
         ...
         '0_hour_c_avg',
         '0_hour_c_std',
         '1_hour_c_avg',
         '1_hour_c_std',
         '2_hour_c_avg',
         '2_hour_c_std',
         ...
         ...
         '23_hour_c_avg',
         '23_hour_c_std',
         '0_hour_d_avg',
         '0_hour_d_std',
         '1_hour_d_avg',
         '1_hour_d_std',
         ...
         ...
         '22_hour_d_avg',
         '22_hour_d_std',
         '23_hour_d_avg',
         '23_hour_d_std’]

у нас получились таблицы :

* hours
* type
* mcc

In [ ]:
mcc.shape

In [ ]:
type.shape

In [ ]:
hours.shape

In [ ]:
361+173+97 -2

In [ ]:
# по дефолту оставляем inner
answer = mcc.merge(type, on='customer_id')\
.merge(hours, on='customer_id')

In [ ]:
answer

In [ ]:
answer.shape

### 21. Какое кол-во столбцов получилось в итоговом DataFrame-е

хочется отметить, что в связи с особенностью подсчета в pivot_table , у некоторых столбцов не считлаоьс std, у которых было всего одно знаечние по уникальному id и mcc. Так что по сути можно добавить, но там будут только 0 и много nan, которые заменим на 0. в 16 задание на примере показал.

In [ ]:
display(f'Кол-во столбцов, которое получилось в новом DF: {answer.shape[1]}')

### 22. Сохранить результирующий датасет в csv-файл features.csv

In [63]:
answer.to_csv('features.csv', index=False)

In [64]:
ls

de_test.ipynb     features.csv      transactions.csv


In [ ]:
pd